# 4. Text data and preprocessing

Most of the recent impressive advances in machine learning and artificial intelligence have emerged in the field of natural language processing. In addition to simpler tasks such as sentiment analysis or text classification, there has been considerable progress in more advanced tasks such as translation and question answering. Text processing involves several unique techniques and methods, which we shall look at next.

## Bag-of-words approach

In natural language processing tasks, the original data is typically in the form of strings, or a list of strings. Neural networks cannot process strings directly, so they need to be converted to arrays consisting of numerical values. This preprocessing usually consists of a succession of distinct steps:

* *Text standardization*: convert everything to lowercase, remove punctuation and other special characters, etc.
* *Tokenization*: split the text into separate units or **tokens** (words, characters, N-grams ...), and set up a vocabulary
* *Vectorization*: associate a numerical vector with each of the tokens in the vocabulary

To investigate an example of implementing these methods, we use the well-known IMDB movie review dataset, that can be downloaded from [the Stanford page of Andrew Maas](https://ai.stanford.edu/~amaas/data/sentiment/). The movie reviews are contained in two folders: one for training samples and one for testing. These folders contain additional subfolders "pos" and "neg" with samples corresponding to their sentiment; note that the train directory contains an extra subfolder "unsup", which is unnecessary for this purpose and should be removed before continuing.   

In [1]:
import keras

batch_size = 32

train_ds = keras.utils.text_dataset_from_directory( 
    '../../aclImdb/train/', 
    validation_split=0.2, 
    subset="training", 
    seed=123,
    batch_size=batch_size)

val_ds = keras.utils.text_dataset_from_directory( 
    '../../aclImdb/train/', 
    validation_split=0.2, 
    subset="validation", 
    seed=123, # same seed as above!
    batch_size=batch_size)

test_ds = keras.utils.text_dataset_from_directory( 
    '../../aclImdb/test/', 
    batch_size=batch_size)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.


Below we take a look at one of the training samples. The integer-valued labels are binary: either zero (negative sentiment) or one (positive sentiment).

In [2]:
for inputs, targets in train_ds:
    print(inputs.shape, targets.shape)
    print(inputs.dtype, targets.dtype)
    print(inputs[0])
    print(targets[0])
    break

(32,) (32,)
<dtype: 'string'> <dtype: 'int32'>
tf.Tensor(b'After, I watched the films... I thought, "Why the heck was this film such a high success in the Korean Box Office?" Even thought the movie had a clever/unusal scenario, the acting wasn\'t that good and the characters weren\'t very interesting. For a Korean movie... I liked the fighting scenes. If you want to watch a film without thinking, this is the film for you. But I got to admit... the film was kind of childish... 6/10', shape=(), dtype=string)
tf.Tensor(1, shape=(), dtype=int32)


For basic text preprocessing, Keras offers a `TextVectorization` layer. This layer can be used to preprocess the string-formed text data into numerical vectors. The code cell below does the following:

* transforms the text to lowercase
* removes punctuation
* does word-level tokenization by splitting on whitespace
* sets up a vocabulary of tokens
* outputs the text converted to a vector of zeroes and ones.  

In [3]:
from keras.layers import TextVectorization

max_tokens = 10000 # Maximum vocabulary size 

vectorization_layer = TextVectorization( 
    max_tokens=max_tokens, 
    output_mode='multi_hot'
)

# Adapt the layer to the text data 
train_texts = train_ds.map(lambda x, y: x) 
vectorization_layer.adapt(train_texts)

# Apply the vectorization to the datasets 
train_ds_bow = train_ds.map(lambda x, y: (vectorization_layer(x), y)) 
val_ds_bow = val_ds.map(lambda x, y: (vectorization_layer(x), y))
test_ds_bow = test_ds.map(lambda x, y: (vectorization_layer(x), y))

After this, the samples drawn from the Datasets are no longer strings, but vectors with as many elements as there are tokens in the vocabulary; in our example, we restrict the vocabulary size to 10000 most commonly encountered tokens in the training texts. Each of the vector elements are either one or zero depending on whether the text contained that particular token or not. 

In [4]:
for inputs, targets in train_ds_bow:
    print(inputs.shape, targets.shape)
    print(inputs.dtype, targets.dtype)
    print(inputs[0])
    print(targets[0])
    break

(32, 10000) (32,)
<dtype: 'int64'> <dtype: 'int32'>
tf.Tensor([1 1 1 ... 0 0 0], shape=(10000,), dtype=int64)
tf.Tensor(0, shape=(), dtype=int32)


This is a very simple example of a *bag-of-words* approach to natural language processing: the word vector merely indicates the presence or absence of a set of words in the input text. Let us now build and train a very simple fully connected classifier for the sentiment analysis task:

In [5]:
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout

model = Sequential([
    Input(shape=(max_tokens,)),
    Dense(units=16, activation='relu'),
    Dropout(0.5),
    Dense(units=1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model.fit(train_ds_bow, epochs=10, validation_data=val_ds_bow)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.8247 - loss: 0.4112 - val_accuracy: 0.8860 - val_loss: 0.2866
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.8922 - loss: 0.2849 - val_accuracy: 0.8860 - val_loss: 0.2970
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9109 - loss: 0.2511 - val_accuracy: 0.8866 - val_loss: 0.3051
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9166 - loss: 0.2436 - val_accuracy: 0.8898 - val_loss: 0.3145
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.9226 - loss: 0.2321 - val_accuracy: 0.8880 - val_loss: 0.3361
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.9269 - loss: 0.2246 - val_accuracy: 0.8892 - val_loss: 0.3457
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.9294 - loss: 0.2213 - val_accuracy: 0.8902 - val_loss: 0.3416
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - accuracy: 0.9309 - loss: 0.2189 - val_accuracy: 

Let us test the classifier:

In [6]:
print(f"Test accuracy: {model.evaluate(test_ds_bow)[1]:.4f}")

782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.8789 - loss: 0.3582
Test accuracy: 0.8789


We can also inspect the vocabulary formed during the text vectorization process. The first element ([UNK]) of the corresponding dictionary is reserved for unknown words not contained in the vocabulary.

In [7]:
vocabulary = vectorization_layer.get_vocabulary()
print(vocabulary[0:20]) # print 20 most common tokens

['[UNK]', np.str_('the'), np.str_('and'), np.str_('a'), np.str_('of'), np.str_('to'), np.str_('is'), np.str_('in'), np.str_('it'), np.str_('i'), np.str_('this'), np.str_('that'), np.str_('br'), np.str_('was'), np.str_('as'), np.str_('with'), np.str_('for'), np.str_('movie'), np.str_('but'), np.str_('film')]


## Word embeddings

Representing texts as multi-hot vectors of zeroes and ones fails to take into account many aspects of natural language: the order of words in the text, the semantic relationships between different words and so on. Also, the vector representation is very sparse (the overwhelming majority of the vector elements are zero) and wasteful. A much better and useful alternative is to use dense encodings of floating-point numbers with smaller dimensionality. This approach is called **word embeddings**.

With word embeddings, each word/token is associated with a vector in a small-dimensional (compared to the vocabulary size) embedding space. The components of these vectors are first initialized randomly, and then treated as trainable parameters. During training, the floating-point values of these vector components change iteratively, as they gather information about the dataset, and the relationships between words. 

In Keras, the word vector components are arranged in a special `Embedding` layer, as shown in the example implementation below. First, we create a new vectorization layer, which now outputs texts as lists of integer-valued token indices, and adapt this to the training texts. Since the text samples are of varying length, we define a `sequence_length` variable to restrict all lists to the same length (either by truncating long samples, or padding the short ones with zeroes).

In [8]:
# Define the TextVectorization layer 
max_tokens = 10000 # Maximum vocabulary size 
sequence_length = 250 # Maximum sequence length 

vectorization_layer = TextVectorization( 
    max_tokens=max_tokens, 
    output_mode='int', 
    output_sequence_length=sequence_length )

train_texts = train_ds.map(lambda x, y: x) 
vectorization_layer.adapt(train_texts)

train_ds_int = train_ds.map(lambda x, y: (vectorization_layer(x), y)) 
val_ds_int = val_ds.map(lambda x, y: (vectorization_layer(x), y))
test_ds_int = test_ds.map(lambda x, y: (vectorization_layer(x), y))

This is how the preprocessed samples now appear: 

In [9]:
for inputs, targets in train_ds_int:
    print(inputs.shape, targets.shape)
    print(inputs.dtype, targets.dtype)
    print(inputs[0])
    print(targets[0])
    break

(32, 250) (32,)
<dtype: 'int64'> <dtype: 'int32'>
tf.Tensor(
[  10  255   11    7    2   88  968   20 4574 1206   44   91    6 1315
   11   20  139   26   75  629    8 2180   17    2 1081    6  656   21
    2  200  288    9    7 1424   21  394   38   94  238   23  162   17
    9    3  824    9   46 1383    1  116    4   85  283   16    2  447
    3    2  869    2  346   37 1547 7555 2639 3245    3    1    1   24
 3676    4 1006    5    2  517    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0 

Now that we have preprocessed our text samples to integer lists of fixed length, they can be directly inserted into a Keras Embedding layer, which converts the integer lists to floating-point valued tensors of shape (sequence length, embedding dimension). 

As can be seen in the cell below, the two-dimensional tensors from the Embedding layer are first converted to one-dimensional ones with shape (sequence length$\cdot$embedding dimension, ) using a Reshape layer. This is then followed up by a fully connected classifier consisting of two Dense layers: one hidden layer plus the final output layer. The Dropout layers are added to reduce overfitting, and the model is trained for ten epochs.      

In [10]:
from keras.layers import Embedding, Dropout, Reshape

embed_dim = 64 # dimension of the word embeddings

model = Sequential([
    Input(shape=(sequence_length,)),
    Embedding(input_dim=max_tokens, output_dim=embed_dim),
    Reshape((sequence_length * embed_dim,)),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(units=1, activation='sigmoid')
])

model.summary()

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.fit(train_ds_int, epochs=10, validation_data=val_ds_int)

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 250, 64)        │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 16000)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 16000)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │       512,032 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,152,065 (4.39 MB)

 Trainable params: 1,152,065 (4.39 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - accuracy: 0.6701 - loss: 0.5813 - val_accuracy: 0.8228 - val_loss: 0.3930
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.8611 - loss: 0.3328 - val_accuracy: 0.8428 - val_loss: 0.3668
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 8s 13ms/step - accuracy: 0.9218 - loss: 0.2053 - val_accuracy: 0.8418 - val_loss: 0.4014
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.9513 - loss: 0.1296 - val_accuracy: 0.8426 - val_loss: 0.4805
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.9711 - loss: 0.0832 - val_accuracy: 0.8372 - val_loss: 0.5687
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - accuracy: 0.9775 - loss: 0.0659 - val_accuracy: 0.8346 - val_loss: 0.6207
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 13s 21ms/step - accuracy: 0.9819 - loss: 0.0534 - val_accuracy: 0.8352 - val_loss: 0.6848
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 13s 20ms/step - accuracy: 0.9851 - loss: 0.0429 - val_a

The validation accuracy has not improved from the much simpler bag-of-words model, which is not very surprising: sentiments in the text can be expected to be reasonably well determined by merely looking at which kinds of words it contains. However, it is easy to deduce that with problems that require deeper insights into the subtle meanings of written text, it is often necessary to use models that are able to preserve the word orderings. 

Also, it is interesting to see that, after training our model, the word vectors in the Embedding layer encode semantic information about the relationships between words. To see that, first we extract the word vector components from the embedding layer, and construct the dictionaries from the vocabulary: 

In [11]:
# Extract the embedding weights from the trained model 
embedding_layer = model.layers[0] # the first layer of the model
embedding_weights = embedding_layer.get_weights()[0] # shape (10000, 64)

# Get the word index from the tokenizer 
vocabulary = vectorization_layer.get_vocabulary() 
index_word = {idx: word for idx, word in enumerate(vocabulary)}
word_index = {word: idx for idx, word in enumerate(vocabulary)}

The following code snippet defines a function that takes a word as an input, and compares its word vector representation to those of all other words in the dictionary in turn, using cosine similarity (essentially the same as taking the dot product of the two vectors). The function then outputs the words with most similar representation.  

In [12]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity 

# Function to find similar words 
def find_similar_words(target_word, top_n=10): 
    if target_word not in word_index: 
        return f'Word "{target_word}" not in vocabulary' 
    target_idx = word_index[target_word] 
    target_embedding = embedding_weights[target_idx].reshape(1, -1) # shape (1, 64)
    similarities = cosine_similarity(embedding_weights, target_embedding).reshape(-1) # shape (10000,)
    similar_indices = np.argsort(similarities)[-top_n-1:-1][::-1] 
    similar_words = [index_word[idx] for idx in similar_indices] 
    return similar_words 
    
word_to_check = 'bad'
print(f'Words similar to "{word_to_check}": {find_similar_words(word_to_check)}')

Words similar to "bad": [np.str_('pathetic'), np.str_('dull'), np.str_('awful'), np.str_('waste'), np.str_('pointless'), np.str_('laughable'), np.str_('worst'), np.str_('boring'), np.str_('dreadful'), np.str_('worse')]


From this output, it is obvious that the model has been able to figure out something about how the words resemble each other.